In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mltable
import os

In [2]:
client = MLClient.from_config(credential=DefaultAzureCredential(), path=".")
ws = client.workspaces.get(name=client.workspace_name)
print(ws.name)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/.azureml/config.json


ml_test


In [54]:
data_asset = client.data.get("nms_itg_240605", version="1")
df = mltable.load(f'azureml:/{data_asset.id}').to_pandas_dataframe()
df.head()

,five_minute_interval,RU,RRC,MAX_UE,AirMAC,PRB,Watt
0,2024-04-28 16:50:00,NUKG33951S,34.0,8.0,NaN,1.51,NaN
1,2024-04-28 16:50:00,NUKG33630S,121.0,12.0,NaN,2.74,NaN
2,2024-04-28 16:50:00,NUKG33241S,72.0,13.0,NaN,1.87,NaN
3,2024-04-28 16:50:00,NUKG33190S,51.0,11.0,NaN,3.13,NaN
4,2024-04-28 16:50:00,NUKG39569S,45.0,19.0,NaN,4.62,NaN


In [55]:
df['RU']=df['RU'].astype(str)
df['RRC']=df['RRC'].fillna(0)
df['MAX_UE']=df['MAX_UE'].fillna(0)
df['AirMAC']=df['AirMAC'].fillna(0)
df['PRB']=df['PRB'].fillna(0)
df['Watt']=df['Watt'].fillna(0)
df.head()

,five_minute_interval,RU,RRC,MAX_UE,AirMAC,PRB,Watt
0,2024-04-28 16:50:00,NUKG33951S,34.0,8.0,0.0,1.51,0.0
1,2024-04-28 16:50:00,NUKG33630S,121.0,12.0,0.0,2.74,0.0
2,2024-04-28 16:50:00,NUKG33241S,72.0,13.0,0.0,1.87,0.0
3,2024-04-28 16:50:00,NUKG33190S,51.0,11.0,0.0,3.13,0.0
4,2024-04-28 16:50:00,NUKG39569S,45.0,19.0,0.0,4.62,0.0


In [72]:
df[df['RU']=='NUKG33630S'].sort_values(['five_minute_interval'])

,five_minute_interval,RU,RRC,MAX_UE,AirMAC,PRB,Watt
6119083,2024-04-13 00:00:00,NUKG33630S,1.0,1.0,0.87,0.81,298.17
7849403,2024-04-13 00:05:00,NUKG33630S,0.0,1.0,0.00,0.80,295.92
7395919,2024-04-13 00:10:00,NUKG33630S,0.0,1.0,0.01,0.80,296.40
6411005,2024-04-13 00:15:00,NUKG33630S,1.0,2.0,0.81,0.81,298.53
7403380,2024-04-13 00:20:00,NUKG33630S,0.0,1.0,0.20,0.80,296.45
...,...,...,...,...,...,...,...
69411,2024-06-05 10:35:00,NUKG33630S,127.0,11.0,172.47,2.49,337.67
92048,2024-06-05 10:40:00,NUKG33630S,100.0,12.0,141.93,2.06,336.78
156837,2024-06-05 10:45:00,NUKG33630S,81.0,15.0,129.16,2.13,337.82
12658,2024-06-05 10:50:00,NUKG33630S,94.0,11.0,0.00,0.00,337.02


In [83]:
df['RU'].unique().size

546

In [74]:
data_folder = './data-small/'
os.makedirs(data_folder, exist_ok=True)
data_path = os.path.join(data_folder, "nmsdatamini.parquet")
df[df['RU']=='NUKG33630S'].sort_values(['five_minute_interval']).to_parquet(data_path, index=False,row_group_size=50, engine="pyarrow")
paths = [{"file": data_path}]
mltp = mltable.from_parquet_files(paths)
mltp.save("./data-small/")

In [75]:
data_folder = './csv-small/'
os.makedirs(data_folder, exist_ok=True)
data_path = os.path.join(data_folder, "nmsdatamini.csv")
df[df['RU']=='NUKG33630S'].sort_values(['five_minute_interval']).to_csv(data_path, index=False)
paths = [{"file": data_path}]
mltc = mltable.from_delimited_files(paths)
mltc.save("./csv-small/")

In [76]:
from azure.ai.ml import automl, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes, TabularTrainingMode
from azure.ai.ml.entities import Data, AmlCompute


In [77]:
compute_cluster = client.compute.get('hwtest')

In [78]:
compute_cluster

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'hwtest', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourceGroups/rgfabricpoc/providers/Microsoft.MachineLearningServices/workspaces/ml_test/computes/hwtest', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/Users/user07', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f82e20c8160>, 'resource_id': None, 'location': 'koreacentral', 'size': 'Standard_E4s_v3', 'min_instances': 0, 'max_instances': 2, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': False, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x7f82e20c89d0>, 'tier': 'dedicated', 'enable_node_public_ip': True, 'subnet': None})

In [ ]:
from azure.ai.ml import automl, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes, TabularTrainingMode
from azure.ai.ml.entities import Data, AmlCompute

In [85]:
data_input = Input(
    type=AssetTypes.MLTABLE,
    path="./data/"
)


In [80]:
train_data = Data(
    path='./csv-small/',
    type=AssetTypes.MLTABLE,
    name='nms-dataset-csv-mini',
    version='4'
)

client.data.create_or_update(train_data)

Uploading csv-small (0.89 MBs): 100%|██████████| 889287/889287 [00:00<00:00, 7489553.65it/s]




Data({'path': 'azureml://subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourcegroups/rgfabricpoc/workspaces/ml_test/datastores/workspaceblobstore/paths/LocalUpload/c68e84d1252e89bd7dbced9469429b7d/csv-small/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': ['nmsdatamini.csv'], 'type': 'mltable', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nms-dataset-csv-mini', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourceGroups/rgfabricpoc/providers/Microsoft.MachineLearningServices/workspaces/ml_test/data/nms-dataset-csv-mini/versions/4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/Users/user07', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f82e20d8fd0>, 'serialize': <msrest.serialization.Serializer object at 0x7f82e20d88b0>, 'ver

In [ ]:
df = pd.read_parquet("../user06/data/004_impute_s1ap_mart/s1ap_1year_sum_mart_impute.parquet")

## Setup AutoML Forecasting Job

In [86]:
# general job parameters
timeout_minutes = 10
trial_timeout_minutes = 5
exp_name = "nms-test"
target_column_name = "Watt"
forecast_horizon=5
time_column_name = "five_minute_interval"
time_series_id='RU'

In [87]:
forecasting_job = automl.forecasting(
    experiment_name=exp_name,
    training_data=data_input,
    compute="hwtest",
    target_column_name=target_column_name,
    primary_metric="NormalizedRootMeanSquaredError",
    n_cross_validations="auto",
    enable_model_explainability=True,
)

In [88]:
# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=120,
    trial_timeout_minutes=30,
    enable_early_termination=True,
    max_concurrent_trials=2,
    max_nodes=2,
)

# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id,
    frequency="5min",
    target_lags="auto",
    target_rolling_window_size='auto',
)

# Training properties are optional
forecasting_job.set_training(blocked_training_algorithms=["ExtremeRandomTrees"])

In [89]:
# Submit the AutoML job
returned_job = client.jobs.create_or_update(
    forecasting_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/Users/user07/data' 'https://mltestkt.blob.core.windows.net/azureml-blobstore-e4a79d34-4e7f-4ac2-b1a6-528dc447c82a/LocalUpload/6b1f51900967aeb9fdfd9cd6c794eb76/data' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


Created job: compute: azureml:hwtest
creation_context:
  created_at: '2024-06-13T09:09:35.568187+00:00'
  created_by: user07
  created_by_type: User
display_name: green_lychee_59t1cn3r4f
experiment_name: nms-test
forecasting:
  feature_lags: none
  forecast_horizon: 5
  frequency: 5min
  seasonality: auto
  short_series_handling_config: auto
  target_aggregate_function: none
  target_lags: auto
  target_rolling_window_size: auto
  time_column_name: five_minute_interval
  time_series_id_column_names: '[''RU'']'
  use_stl: none
id: azureml:/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourceGroups/rgfabricpoc/providers/Microsoft.MachineLearningServices/workspaces/ml_test/jobs/green_lychee_59t1cn3r4f
limits:
  enable_early_termination: true
  max_concurrent_trials: 2
  max_cores_per_trial: -1
  max_nodes: 2
  max_trials: 1000
  timeout_minutes: 120
  trial_timeout_minutes: 30
log_verbosity: info
n_cross_validations: auto
name: green_lychee_59t1cn3r4f
outputs: {}
primary_metric: no

In [90]:
client.jobs.stream(returned_job.name)

RunId: green_lychee_59t1cn3r4f
Web View: https://ml.azure.com/runs/green_lychee_59t1cn3r4f?wsid=/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourcegroups/rgfabricpoc/workspaces/ml_test
